In [47]:
!pwd

/root/NER-project/code/training_pipeline/notebooks


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [48]:
!pip install -q evaluate seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [49]:
from pathlib import Path
from datasets import load_dataset
import pandas as pd
import torch
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, pipeline
import evaluate
import numpy as np

from huggingface_hub import notebook_login
import wandb

In [50]:
DATA_DIR = Path("./data/wnut_17")
MODEL_DIR = Path("NER_model")

In [51]:
from datasets import load_dataset
wnut = load_dataset("wnut_17", cache_dir=DATA_DIR)

In [52]:
train_df = pd.DataFrame(wnut['train'])
train_df.tokens.apply(len).max()

41

In [53]:
label_list = wnut["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

In [54]:
tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')

In [55]:
example = wnut["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words = True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

In [56]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [57]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [58]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [59]:
import evaluate
seqeval = evaluate.load("seqeval")

In [60]:
labels = [label_list[i] for i in example[f"ner_tags"]]
labels

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-location',
 'I-location',
 'I-location',
 'O',
 'B-location',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [61]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
}

In [62]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
    }
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

In [63]:
model = AutoModelForTokenClassification.from_pretrained(
    'distilbert/distilbert-base-uncased',
    num_labels = 13,
    id2label = id2label,
    label2id= label2id,
    torchscript=True
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [64]:
import os
os.environ["WANDB_PROJECT"]="NER"
os.environ["WANDB_LOG_MODEL"]="end"
os.environ["WANDB_NOTEBOOK_NAME"]="POC"

In [65]:
import wandb
wandb.login()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find POC.
wandb: Currently logged in as: phamlehoangvu97 (thomas24). Use `wandb login --relogin` to force relogin


True

In [66]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "NER_model",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size=16,
    num_train_epochs = 2,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
    report_to="wandb",
    logging_steps=1,
    push_to_hub=True
)

/root/anaconda3/envs/training-pipeline/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [67]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    train_dataset = tokenized_wnut["train"],
    eval_dataset = tokenized_wnut["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
    args = training_args
)

In [68]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.522500,0.288938,0.440239,0.204819,0.279570,0.936642
2,0.061000,0.277326,0.523126,0.303985,0.384525,0.941174


/root/anaconda3/envs/training-pipeline/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/anaconda3/envs/training-pipeline/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=426, training_loss=0.21497215444720547, metrics={'train_runtime': 40.234, 'train_samples_per_second': 168.713, 'train_steps_per_second': 10.588, 'total_flos': 91781128898820.0, 'train_loss': 0.21497215444720547, 'epoch': 2.0})

In [69]:
wandb.finish()

eval/accuracy,▁█
eval/f1,▁█
eval/loss,█▁
eval/precision,▁█
eval/recall,▁█
eval/runtime,▁█
eval/samples_per_second,█▁
eval/steps_per_second,█▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,█▅▂▁▁▂▁▂▂▂▂▁▁▁▂▂▂▂▁▂▂▁▁▃▁▂▁▂▂▂▂▂▂▃▁▁▂▁▂▃


In [90]:
!pip install -q optimum onnx

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [96]:
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
dummy_input = tokenizer(text, return_tensors="pt")

In [97]:
from transformers import AutoTokenizer

model_checkpoint = "NER_model"
save_directory = "onnx/"

# Load a model from transformers and export it to ONNX
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [102]:
torch.onnx.export(
    model, 
    tuple(dummy_input.values()),
    f="onnx/torch-model.onnx",  
    input_names=['input_ids', 'attention_mask'], 
    output_names=['logits'], 
    dynamic_axes={'input_ids': {0: 'batch_size', 1: 'sequence'}, 
                  'attention_mask': {0: 'batch_size', 1: 'sequence'}, 
                  'logits': {0: 'batch_size', 1: 'sequence'}}, 
    do_constant_folding=True, 
    opset_version=13, 
)

In [108]:
import onnx
model_onnx = onnx.load("onnx/torch-model.onnx")
onnx.checker.check_model(model_onnx)

In [112]:
!pip install onnxruntime

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached onnxruntime-1.18.1-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (6.8 MB)


In [ ]:
text2 = "My name is Tom, I live in New York and my girlfriend's name is Elaine. Our parents live in Viet Nam, Nha Trang city, and their names are Que and Mai"
text2 = tokenizer(text2, return_tensors='pt')

In [153]:
import onnxruntime as ort
import numpy as np
ort_sess = ort.InferenceSession('onnx/torch-model.onnx')
outputs = ort_sess.run(None, {'input_ids': text2['input_ids'].numpy(),
                            'attention_mask':  text2['attention_mask'].numpy()})

label = np.argmax(outputs, axis=3)
pred_labels = [label_list[i] for i in label[0][0]]
pred_labels

['O',
 'O',
 'O',
 'O',
 'B-person',
 'O',
 'O',
 'O',
 'O',
 'B-location',
 'B-location',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-person',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-location',
 'B-location',
 'O',
 'B-location',
 'O',
 'B-location',
 'B-location',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-person',
 'O',
 'O',
 'O']